check the qurey 

In [61]:
import pyvo
import configparser
from speedysedfit.photometry_query import get_coordinate

In [63]:
get_coordinate?

Signature: get_coordinate(objectname)
Docstring: <no docstring>
File:      ~/anaconda3/lib/python3.7/site-packages/speedysedfit-0.1.0-py3.7.egg/speedysedfit/photometry_query.py
Type:      function


In [48]:
object_name = 'BD-7_5977'
ra, dec = get_coordinate(object_name)
print(f'ra={ra}; dec={dec}')

ra=349.44494999999995; dec=-6.4752616666666665


In [51]:
tap_info = configparser.ConfigParser()
tap_info.optionxform = str # make sure the options are case sensitive
tap_info.read_file(open('../speedysedfit/'+'/tap_cats_phot.cfg'))
catalogs = tap_info.sections()

#catalog = catalogs[0]
catalog_s = 'https://api.skymapper.nci.org.au/public/tap/'
catalog = 'http://api.skymapper.nci.org.au/public/tap/'
service = pyvo.dal.TAPService(catalog)
table = tap_info.get(catalog, 'table')

In [70]:
from astroquery.vizier import Vizier
from speedysedfit.photometry_query import zpt
from astropy import units as u
def get_parallax(objectname, radius=5):

    v_gaia = Vizier(columns=["Plx", "e_Plx", '+_r', 'Gmag', 'nueff', 'pscol', 'ELAT', 'Solved'])

    data = v_gaia.query_object(objectname, catalog=['I/350/gaiaedr3'], radius=radius*u.arcsec)

    if len(data) == 0:
        return None, None

    data = data['I/350/gaiaedr3'][0]

    plx, plx_e = data['Plx'], data['e_Plx']

    if not data['pscol']:
        data['pscol'] = 0

    # apply parallax zero point correction of Lindgren. If not possible, use the average offset.
    # https://arxiv.org/pdf/2012.01742.pdf
    # https://gitlab.com/icc-ub/public/gaiadr3_zeropoint
    try:
        zp = zpt.get_zpt(data['Gmag'], data['nueff'], data['pscol'], data['ELAT'], data['Solved'])
    except Exception:
        warnings.warn("Could not calculate the parallax zero point offset based on Lindgren+2020, using average")
        zp = 0.02
    #plx -= zp

    return plx, plx_e, zp

objectname = 'J192054.5-200135.56'
plx, plx_e, zp = get_parallax(objectname, radius=5)
print(f'plx={plx}; plx_e={plx_e}; zp={zp}')

plx=0.1123; plx_e=0.06859999895095825; zp=-0.00504


In [14]:
keywords = ""
bands = []
for band in tap_info.options(catalog):
    if 'table' in band: continue
    if 'rakw' in band: continue
    if 'deckw' in band: continue
    if '_unit' in band: continue
    if '_err' in band: continue
    if 'bibcode' in band: continue
    errkw = tap_info.get(catalog, band + '_err') if tap_info.has_option(catalog, band + '_err') else 'e_' + band
    keywords += band + ", " + errkw + ", "
    bands.append(band)                                                                                                     
if keywords[-2:] == ", ":
    keywords = keywords[0:-2]
rakw = tap_info.get(catalog, 'rakw') if tap_info.has_option(catalog, 'rakw') else 'raj2000'
deckw = tap_info.get(catalog, 'deckw') if tap_info.has_option(catalog, 'deckw') else 'dej2000'


In [36]:
query = """SELECT 
     DISTANCE(POINT('ICRS', {rakw:}, {deckw}),
              POINT('ICRS', {ra:}, {dec:})) AS dist, {kws:}
     FROM {table:} AS m
     WHERE 
        1=CONTAINS(POINT('ICRS', {rakw:}, {deckw}),
                 CIRCLE('ICRS', {ra:}, {dec:}, 0.005 ))
     ORDER BY dist""".format(ra=ra, dec=dec, table=table, rakw=rakw, deckw=deckw, kws=keywords)


In [22]:
print(query)

SELECT 
     DISTANCE(POINT('ICRS', raj2000, dej2000),
              POINT('ICRS', 349.44494999999995, -6.4752616666666665)) AS dist, u_psf, e_u_psf, v_psf, e_v_psf, g_psf, e_g_psf, r_psf, e_r_psf, i_psf, e_i_psf, z_psf, e_z_psf
     FROM dr1.master AS m
     WHERE 
        1=CONTAINS(POINT('ICRS', raj2000, dej2000),
                 CIRCLE('ICRS', 349.44494999999995, -6.4752616666666665, 0.005 ))
     ORDER BY dist


In [42]:
query1 = """SELECT 
TOP 10 
{columns}
FROM gaiadr2.gaia_source
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2
"""

In [27]:
results = service.run_sync(query).to_table()

DALQueryError: Missing REQUEST paramater.

In [30]:
from astroquery.utils.tap import TapPlus

In [29]:
import astroquery

In [52]:
server = TapPlus(url=catalog_s,                                                                  
                                      server_context="data-server",
                                      tap_context="tap",
                                      upload_context="Upload",
                                      table_edit_context="TableTool",
                                      data_context="data",
                                      datalink_context="datalink")

Created TAP+ (v1.2.1) - Connection:
	Host: api.skymapper.nci.org.au
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [32]:
catalog

'http://api.skymapper.nci.org.au/public/tap/'

In [34]:
server1 = TapPlus(url="https://geadata.esac.esa.int/",                                                                  
                                      server_context="data-server",
                                      tap_context="tap",
                                      upload_context="Upload",
                                      table_edit_context="TableTool",
                                      data_context="data",
                                      datalink_context="datalink")

Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [43]:
job = server1.launch_job_async(query1)

404 Error 404:
Job '1630056080697ODL' not found.


HTTPError: Error 404:
Job '1630056080697ODL' not found.

In [53]:
job = server.launch_job_async(query)

HTTPError: Not Found

In [54]:
catalogs

['http://api.skymapper.nci.org.au/public/tap/',
 'http://vao.stsci.edu/PS1DR2/TapService.aspx/']

In [55]:
http://vao.stsci.edu/ps1dr2/querymanager.html

SyntaxError: invalid syntax (<ipython-input-55-78c88ce36e0e>, line 1)

In [56]:
%matplotlib inline
from astropy.io import ascii
from astropy.table import Table

import sys
import re
import numpy as np
import pylab
import json
import requests

try: # Python 3.x
    from urllib.parse import quote as urlencode
    from urllib.request import urlretrieve
except ImportError:  # Python 2.x
    from urllib import pathname2url as urlencode
    from urllib import urlretrieve

try: # Python 3.x
    import http.client as httplib 
except ImportError:  # Python 2.x
    import httplib 

In [57]:
def ps1cone(ra,dec,radius,table="mean",release="dr1",format="csv",columns=None,
           baseurl="https://catalogs.mast.stsci.edu/api/v0.1/panstarrs", verbose=False,
           **kw):
    """Do a cone search of the PS1 catalog
    
    Parameters
    ----------
    ra (float): (degrees) J2000 Right Ascension
    dec (float): (degrees) J2000 Declination
    radius (float): (degrees) Search radius (<= 0.5 degrees)
    table (string): mean, stack, or detection
    release (string): dr1 or dr2
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'nDetections.min':2)
    """
    
    data = kw.copy()
    data['ra'] = ra
    data['dec'] = dec
    data['radius'] = radius
    return ps1search(table=table,release=release,format=format,columns=columns,
                    baseurl=baseurl, verbose=verbose, **data)

def ps1search(table="mean",release="dr1",format="csv",columns=None,
           baseurl="https://catalogs.mast.stsci.edu/api/v0.1/panstarrs", verbose=False,
           **kw):
    """Do a general search of the PS1 catalog (possibly without ra/dec/radius)
    
    Parameters
    ----------
    table (string): mean, stack, or detection
    release (string): dr1 or dr2
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'nDetections.min':2).  Note this is required!
    """
    
    data = kw.copy()
    if not data:
        raise ValueError("You must specify some parameters for search")
    checklegal(table,release)
    if format not in ("csv","votable","json"):
        raise ValueError("Bad value for format")
    url = "{baseurl}/{release}/{table}.{format}".format(**locals())
    if columns:
        # check that column values are legal
        # create a dictionary to speed this up
        dcols = {}
        for col in ps1metadata(table,release)['name']:
            dcols[col.lower()] = 1
        badcols = []
        for col in columns:
            if col.lower().strip() not in dcols:
                badcols.append(col)
        if badcols:
            raise ValueError('Some columns not found in table: {}'.format(', '.join(badcols)))
        # two different ways to specify a list of column values in the API
        # data['columns'] = columns
        data['columns'] = '[{}]'.format(','.join(columns))

# either get or post works
#    r = requests.post(url, data=data)
    r = requests.get(url, params=data)

    if verbose:
        print(r.url)
    r.raise_for_status()
    if format == "json":
        return r.json()
    else:
        return r.text


def checklegal(table,release):
    """Checks if this combination of table and release is acceptable
    
    Raises a VelueError exception if there is problem
    """
    
    releaselist = ("dr1", "dr2")
    if release not in ("dr1","dr2"):
        raise ValueError("Bad value for release (must be one of {})".format(', '.join(releaselist)))
    if release=="dr1":
        tablelist = ("mean", "stack")
    else:
        tablelist = ("mean", "stack", "detection")
    if table not in tablelist:
        raise ValueError("Bad value for table (for {} must be one of {})".format(release, ", ".join(tablelist)))


def ps1metadata(table="mean",release="dr1",
           baseurl="https://catalogs.mast.stsci.edu/api/v0.1/panstarrs"):
    """Return metadata for the specified catalog and table
    
    Parameters
    ----------
    table (string): mean, stack, or detection
    release (string): dr1 or dr2
    baseurl: base URL for the request
    
    Returns an astropy table with columns name, type, description
    """
    
    checklegal(table,release)
    url = "{baseurl}/{release}/{table}/metadata".format(**locals())
    r = requests.get(url)
    r.raise_for_status()
    v = r.json()
    # convert to astropy table
    tab = Table(rows=[(x['name'],x['type'],x['description']) for x in v],
               names=('name','type','description'))
    return tab


def mastQuery(request):
    """Perform a MAST query.

    Parameters
    ----------
    request (dictionary): The MAST request json object

    Returns head,content where head is the response HTTP headers, and content is the returned data
    """
    
    server='mast.stsci.edu'

    # Grab Python Version 
    version = ".".join(map(str, sys.version_info[:3]))

    # Create Http Header Variables
    headers = {"Content-type": "application/x-www-form-urlencoded",
               "Accept": "text/plain",
               "User-agent":"python-requests/"+version}

    # Encoding the request as a json string
    requestString = json.dumps(request)
    requestString = urlencode(requestString)
    
    # opening the https connection
    conn = httplib.HTTPSConnection(server)

    # Making the query
    conn.request("POST", "/api/v0/invoke", "request="+requestString, headers)

    # Getting the response
    resp = conn.getresponse()
    head = resp.getheaders()
    content = resp.read().decode('utf-8')

    # Close the https connection
    conn.close()

    return head,content


def resolve(name):
    """Get the RA and Dec for an object using the MAST name resolver
    
    Parameters
    ----------
    name (str): Name of object

    Returns RA, Dec tuple with position"""

    resolverRequest = {'service':'Mast.Name.Lookup',
                       'params':{'input':name,
                                 'format':'json'
                                },
                      }
    headers,resolvedObjectString = mastQuery(resolverRequest)
    resolvedObject = json.loads(resolvedObjectString)
    # The resolver returns a variety of information about the resolved object, 
    # however for our purposes all we need are the RA and Dec
    try:
        objRa = resolvedObject['resolvedCoordinate'][0]['ra']
        objDec = resolvedObject['resolvedCoordinate'][0]['decl']
    except IndexError as e:
        raise ValueError("Unknown object '{}'".format(name))
    return (objRa, objDec)

In [59]:
pwd

'/share/lijiao/Documents/speedysedfit/ipynb'